# Test Notebook

Data: 9.12.20

In [ ]:
%pylab inline
#plt.style.use('dark_background')

In [ ]:
import pyopencl as cl
cl_platform = cl.get_platforms()[0]
cl_device = cl_platform.get_devices(cl.device_type.GPU)[0]
cl_context = cl.Context([cl_device])
cl_queue = cl.CommandQueue(cl_context)

In [ ]:
from pyopencl.tools import MemoryPool, ImmediateAllocator
cl_allocator = None #MemoryPool(ImmediateAllocator(cl_queue))

In [ ]:
import copy
import importlib
import os.path

In [ ]:
import fieldretriever
importlib.reload(fieldretriever)

In [ ]:
%run dataloader

In [ ]:
def create_circular_mask(shape, center=(0,0), radius=1.):
    
    x = linspace(-1,1,shape[1])
    y = linspace(-1,1,shape[0])
                   
    X, Y = np.meshgrid(x,y, sparse=True)
    R2 = np.square(X - center[0]) + np.square(Y - center[1])
    mask = R2 <= radius**2
    return mask#.astype(float)

# MESSUNGEN VOM ..... 

## Data Loader

In [ ]:
folder = 'Z:/groups/Tweezers/data/2020-12-09/'

DLoader = DataLoader(folder = folder, name='Forces')

### Load dark frame images

In [ ]:
nr_bg = 21
bfp_bg, __ = DLoader.load_bg_dark_images(nr = nr_bg)


## Mask

### Loading focal mask and flat field data

In [ ]:
m_nr_mask = 23
I_mask, I_spots, pos_spots, I0 = DLoader.load_I0_and_focal_mask_data(nr = m_nr_mask)

In [ ]:
imshow((I0-1*bfp_bg), vmin=0.01, vmax=None)
colorbar()

In [ ]:
I0[:,512].mean()/I0[512,:].mean()

In [ ]:
figure()
imshow(I_mask.T[:,::-1] + 1*I_spots.T[:,::-1])
colorbar()

In [ ]:
%run create_mask

In [ ]:
M = CreateMaskFocal(img_mask = I_mask, img_spots = I_spots, pos_spots_holo = pos_spots, field_shape = (1024,) * 2)

M.create_mask(sigma = 0.3, gamma = .6, dpi = 80, figsize = (12,3)) #sigma is most important to change if not all pixels are detected!

In [ ]:
fourier_mask = M.mask.copy()
fourier_plane_size = M.field_size
fourier_plane_shape = M.field_shape
print('Fourier plane size = %.3f mm' %(fourier_plane_size[0]*1e3))

## circular mask for $I_0$

In [ ]:
# I0 mit Hintergrundsubtraktion und runder Maske
palette = copy.copy(plt.cm.viridis)
palette.set_over('r', 1.0)
palette.set_under('w', 0.0)

mask_I0 = create_circular_mask(I0.shape, radius=0.910, center=(0.001, 0.00)) #08.04.20

I0_t_masked = clip ( (I0 - bfp_bg)*mask_I0, 0 , None)

figure(dpi = 150)
imshow( I0_t_masked, vmin=0.01, cmap=palette)
axis('off')
colorbar()

# Fieldretrieval for a single BFP image

### load BFP and SLM pattern

In [ ]:
nr_meas = 15 # WH 16 und empty 17
nr_img = 10
bfp = DLoader.load_bfp(nr_meas, nr_img)

In [ ]:
figure(dpi = 100)
imshow(bfp, cmap = cm.gray, interpolation = 'lanczos', vmin = 0, vmax = 25)
axis('off')

### Load data

In [ ]:
# load data

bfp, phase_slm, phase_slm0 = DLoader.load_data(nr_meas, idx=nr_img, 
                                                  #normalize_exposure = False
                                                 )
mask_bfp = create_circular_mask(bfp.shape, radius = 1.02)
imshow((bfp-bfp_bg)*mask_bfp, vmin=0.01, cmap=palette)
colorbar()

trap_pos = DLoader.load_trap_positions(nr_meas, nr_img)

# select active traps, scale positions, swap x/y
trap_pos = trap_pos[0:-1, [1,0,2]] * 1e-6 * 1.173   #TODO: z-scaling different?

mask_bfp = create_circular_mask(bfp.shape)
bfp_measured = ((bfp-bfp_bg)*mask_bfp).astype(float32)



### Retrieve field

In [ ]:
scale_I0 =  1.2*0.934 #compensating that the fringer (model of slm behaviour) already accounts for reduced diffraction efficiency

PSF_detector = fieldretriever.double_gaussian_kernel(
    N=1024, sigma_1=.9, sigma_2=40., p2=0.12).astype(float32)

R = fieldretriever.FieldRetrieverGPU(cl_context=cl_context, cl_queue=cl_queue, cl_allocator=cl_allocator,
              slm_phase = phase_slm,
              slm_phase0 = phase_slm0,
              I0 = I0_t_masked * scale_I0,
              R0 = 1.02,
              fourier_plane_size = fourier_plane_size,
              fourier_plane_shape = fourier_plane_shape,
              object_plane_shape = fourier_plane_shape,
              fourier_plane_mask = fourier_mask,
              detector_psf = PSF_detector)

R.init_all(pos=trap_pos, patch_size = 5e-6)


In [ ]:
figure(dpi = 100)
imshow(log10(R.object_multiareafield.field.intensity),vmin = 0.1, vmax = 8, zorder = 0)
title('object plane')

In [ ]:
iterations = 0
stepsize = 100
momentum = 0.85 #0.8
retrieved_empty, retrieved_empty_no_transmission = R.retrieve_field(bfp_measured, 
                               iterations = iterations, stepsize = stepsize, momentum = momentum)

print('-------   if scale I0 good this should be about 1 --------')
#print(bfp_measured.sum()/retrieved_empty.sum())
print(bfp_measured.sum()/retrieved_empty.sum())

In [ ]:
iterations = 100
stepsize = 100 
momentum = 0.85

retrieved0, retrieved_no_transmission = R.retrieve_field(bfp_measured, 
                               iterations = iterations, stepsize = stepsize, momentum = momentum)

In [ ]:
log0 = R.log.copy()
plot(log0)

In [ ]:
retrieved = retrieved0#_c
original = (bfp - bfp_bg)*R.mask_bfp

print(retrieved.sum())
print(original.sum())
print(1/retrieved.sum()*original.sum())

v_min = 0
v_max = max(original.max(), retrieved.max())
v_max_residuum = 0.2 * v_max

r, c = slice(None, None), slice(None, None)
#r, c = slice(400,600), slice(400,600)

fig, (a1,a2,a3) = subplots(1,3, figsize=(12,4))
a1.imshow(original[r,c], vmin = v_min, vmax = v_max)
a2.imshow(retrieved[r,c], vmin = v_min, vmax = v_max)
a3.imshow((original - retrieved)[r,c], vmin = -v_max_residuum, vmax = v_max_residuum, cmap = cm.RdBu_r)


fig.tight_layout()

In [ ]:
K = 512
s0 = slice(None, K)
s1 = slice(K, None)
s00 = s0, s0
s01 = s0, s1
s10 = s1, s0
s11 = s1, s1

d = original.copy()
# d = zeros_like(original) # for checking
d[s01] = retrieved[s01]
#d[s10] = retrieved[s10]
xp, yp = [], []
for n in range(512, 1024,16)[::2]:
    s = slice(n, n+16), s0
    d[s] = retrieved[s]
    xp.extend([0, K-.5, NaN, 0, K-.5, NaN])
    yp.extend([n-.5, n-.5, NaN, n+16-.5, n+16-.5, NaN])
    
d[s11] = (original - retrieved)[s11]*-2 + v_max/2

figure(dpi=100, figsize=(10,10))
imshow(d, vmin=0, vmax=0.8*v_max, cmap = plt.cm.RdBu_r, interpolation='lanczos')
plot([0, 1024, NaN, 511.5, 511.5], [511.5, 511.5, NaN, 0, 1024], 'w-', lw=.5, alpha=.5)
plot(xp, yp, 'w-', lw=.5, alpha=.5)
#axis('off')

xlim(200, 800)
ylim(700, 300)
#axis('image')

In [ ]:
R.calculate_individual_farfields( just_intesities = True)
#R_c.calculate_individual_farfields()

In [ ]:
# ind farfield comparison
ind_bfps = DLoader.load_bfps_single_trap(nr = nr_meas, idx = nr_img)
v_max = 0.4*ind_bfps.max()

fig, ax = subplots(2,len(R.object_subfields_0), sharex=True, sharey=True, squeeze=True, tight_layout = True)
for i, bfps in enumerate(ind_bfps):
    a = ax[:,i]
    a[0].imshow(R.individual_farfield_intensities[i], vmin=0, vmax=1/ind_bfps.shape[0]*0.85*v_max)    
    im = a[1].imshow(bfps, vmin=0, vmax=v_max)
    for aa in a:
        aa.axis('off')
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
#fig.colorbar(im, cax=cbar_ax)

In [ ]:
# calculate force from retrieved individual bfps

In [ ]:
%run force

In [ ]:
C = CalculateForce(R0 = R.R0, scale_au_to_pN = 1/45000)

In [ ]:
F_ind = array(([C.calc_force(I) for I in R.individual_farfield_intensities]))

In [ ]:
F_ind